In [ ]:
import warnings
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    
    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']   = (Src['Close'] - Src['Open'])
    Src['Hilo']     = (Src['High']  - Src['Low'])
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Date'], sort=0, group_keys=0)

    # ================ Day Indicators ================ #
    Calc['Day Open']    = Calc_by['Open'].transform('first')           .reset_index(drop=1)
    Calc['Day High']    = Calc_by['High'].transform('max')             .reset_index(drop=1)
    Calc['Day Low']     = Calc_by['Low'].transform('min')              .reset_index(drop=1)
    Calc['Day Close']   = Calc_by['Close'].transform('last')           .reset_index(drop=1)

    Calc['Day Hilo']    = Calc['Day High']  - Calc['Day Low']
    Calc['Day Change']  = Calc['Day Close'] - Calc['Day Open']
    Calc['Day Chg Abs'] = Calc['Day Chg'].abs()


    # ================ Current Indicators ================ #
    Calc['T']       = Calc.groupby('Date', sort=0)['Time'].rank()
    
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Avg'] = Calc['Chg Abs'].expanding().mean()


    # ================ Current Signals ================ #
    Calc['Change Pos']      = (Calc['Change']  >= 0)
    Calc['Change Neg']      = (Calc['Change']  <  0)
    Calc['Chg Abs > Avg']   = (Calc['Chg Abs'] >  Calc['Chg Avg'])

    # ================ Forward Indicators ================ #
    FUTS = [1, 2]
    for x in FUTS:  Calc[f'Hilo +{x}']       =  Calc.groupby('Date', sort=0)[f'Hilo'].shift(-x)
    for x in FUTS:  Calc[f'Change +{x}']     =  Calc.groupby('Date', sort=0)[f'Change'].shift(-x)
    
    # ================ Forward Signals ================ #
    for x in FUTS:  Calc[f'Change +{x} Pos'] = (Calc[f'Change +{x}'] >= 0)
    for x in FUTS:  Calc[f'Change +{x} Neg'] = (Calc[f'Change +{x}'] <  0)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [ ]:
COND_BULL = (Calc['T']==1) & Calc['Chg Abs > Avg'] & Calc['Change Pos']
COND_BEAR = (Calc['T']==1) & Calc['Chg Abs > Avg'] & Calc['Change Neg']

In [ ]:
pd.DataFrame([
    { 'Condition':'Change Pos', **Calc[COND_BULL][['Change +1 Pos','Change +1 Neg']].mean() }, 
    { 'Condition':'Change Neg', **Calc[COND_BEAR][['Change +1 Pos','Change +1 Neg']].mean() }, 
])

In [ ]:
pd.DataFrame([
    { 'Condition':'Change Pos', **Calc[COND_BULL]['Change +1'].describe().T },
    { 'Condition':'Change Neg', **Calc[COND_BEAR]['Change +1'].describe().T },
])

In [ ]:
pd.DataFrame([
    { 'Condition':'Change Pos', **Calc[COND_BULL]['Hilo +1'].describe().T },
    { 'Condition':'Change Neg', **Calc[COND_BEAR]['Hilo +1'].describe().T },
])

# Snippets

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['T'] = Calc.groupby('Date', sort=0)['Time'].rank()

    # ================ Current Indicators ================ #
    Calc['Change']  = (Calc['Close'] - Calc['Open'])
    Calc['Hilo']    = (Calc['High']  - Calc['Low'])

    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Avg'] = Calc['Chg Abs'].expanding().mean()

    # ================ Current Signals ================ #
    Calc['Change Pos']      = (Calc['Change']  >= 0)
    Calc['Change Neg']      = (Calc['Change']  <  0)
    Calc['Chg Abs > Avg']   = (Calc['Chg Abs'] >  Calc['Chg Avg'])

    # ================ Forward Indicators ================ #
    FUTS = [1, 2]
    for x in FUTS:  Calc[f'Hilo +{x}']       =  Calc.groupby('Date', sort=0)[f'Hilo'].shift(-x)
    for x in FUTS:  Calc[f'Change +{x}']     =  Calc.groupby('Date', sort=0)[f'Change'].shift(-x)
    
    # ================ Forward Signals ================ #
    for x in FUTS:  Calc[f'Change +{x} Pos'] = (Calc[f'Change +{x}'] >= 0)
    for x in FUTS:  Calc[f'Change +{x} Neg'] = (Calc[f'Change +{x}'] <  0)
    return Calc

In [ ]:
COND_BULL = (Calc['T']==1) & Calc['Chg Abs > Avg'] & Calc['Change Pos']
COND_BEAR = (Calc['T']==1) & Calc['Chg Abs > Avg'] & Calc['Change Neg']

pd.DataFrame([
    { 'Condition':'Change Pos', **Calc[COND_BULL][['Change +1 Pos','Change +1 Neg']].mean() }, 
    { 'Condition':'Change Neg', **Calc[COND_BEAR][['Change +1 Pos','Change +1 Neg']].mean() }, 
])

pd.DataFrame([
    { 'Condition':'Change Pos', **Calc[COND_BULL]['Change +1'].describe().T },
    { 'Condition':'Change Neg', **Calc[COND_BEAR]['Change +1'].describe().T },
])

pd.DataFrame([
    { 'Condition':'Change Pos', **Calc[COND_BULL]['Hilo +1'].describe().T },
    { 'Condition':'Change Neg', **Calc[COND_BEAR]['Hilo +1'].describe().T },
])